# Imports and definitions

In [70]:
from pathlib import Path
import time

import polars as pl
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve, f1_score

from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

import optuna

_ = pl.Config.set_tbl_cols(None)
_ = pl.Config.set_fmt_str_lengths(500)
_ = pl.Config.set_fmt_float("full")

In [2]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning, module='sklearn')

In [3]:
base_dir = Path('/Users/danlab/code/magenta-task/')
code_dir = base_dir / 'notebooks'
data_dir = code_dir / "data"
features_dir = data_dir / 'features'
train_dir = data_dir / 'train'
db_dir = 'sqlite:///data/models/{}.db'

# Load data

In [4]:
%%time

train = pl.read_parquet(train_dir / 'data-v0-80.parquet')

CPU times: user 24 ms, sys: 13.5 ms, total: 37.5 ms
Wall time: 39.1 ms


# Prepare data

In [49]:
age_b_1 = train.filter(pl.col("age") < 55)
age_b_2 = train.filter(pl.col("age") >= 55)

In [50]:
X_1 = age_b_1.select(pl.exclude(['rating_account_id', 'customer_id', 'has_done_upselling', 'age']))
y_1 = age_b_1.select('has_done_upselling')

X_2 = age_b_2.select(pl.exclude(['rating_account_id', 'customer_id', 'has_done_upselling', 'age']))
y_2 = age_b_2.select('has_done_upselling')


In [51]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define objectives

In [52]:
# Compute the ratio of negative to positive instances in the target
ratio_negative_to_positive_b_1 = (
    (y_1['has_done_upselling'] == False).sum() / (y_1['has_done_upselling'] == True).sum()
)
print("ratio_negative_to_positive_b_1:", ratio_negative_to_positive_b_1)

ratio_negative_to_positive_b_1: 12.750193199381762


In [53]:
# Compute the ratio of negative to positive instances in the target
ratio_negative_to_positive_b_2 = (
    (y_2['has_done_upselling'] == False).sum() / (y_2['has_done_upselling'] == True).sum()
)
print("ratio_negative_to_positive_b_1:", ratio_negative_to_positive_b_2)

ratio_negative_to_positive_b_1: 18.069114470842333


In [54]:
def find_optimal_f1(valid_y, preds):
    # Find optimal threshold for F1
    precision, recall, thresholds = precision_recall_curve(valid_y, preds)
    f1_scores_thresh = 2 * (precision * recall) / (precision + recall + 1e-8)
    optimal_idx = np.argmax(f1_scores_thresh)
    optimal_threshold = thresholds[optimal_idx] if optimal_idx < len(thresholds) else 0.5
    
    # Make binary predictions using optimal threshold
    pred_labels = (preds >= optimal_threshold).astype(int)
    return f1_score(valid_y, pred_labels), optimal_threshold

In [55]:
def xgboost_objective(trial, X, y, skf, ratio_negative_to_positive, n_splits=5):
    '''
    XGBoost objective function using stratified cross-validation
    
    Args:
        trial: Optuna trial object
        n_splits: Number of folds for cross-validation (default: 5)
    '''
    
    cv_scores = []
    optimal_thresholds = []

    param = {
        'verbosity': 0,
        'n_jobs': 4,
        'early_stopping_rounds': 16,
        'eval_metric': 'aucpr',
        'scale_pos_weight': trial.suggest_float("scale_pos_weight", 1.0, ratio_negative_to_positive * 1.5, log=True),
        'objective': 'binary:logistic',
        'tree_method': 'hist',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),

        # L2 regularization weight.
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        # L1 regularization weight.
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),
        # sampling according to each tree.
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
    }

    if param['booster'] in ['gbtree', 'dart']:
        # maximum depth of the tree, signifies complexity of the tree.
        param['max_depth'] = trial.suggest_int('max_depth', 3, 20)
        # minimum child weight, larger the term more conservative the tree.
        param['min_child_weight'] = trial.suggest_int('min_child_weight', 2, 10)
        param['eta'] = trial.suggest_float('eta', 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param['gamma'] = trial.suggest_float('gamma', 1e-8, 1.0, log=True)
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])

    if param['booster'] == 'dart':
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_float('rate_drop', 1e-8, 1.0, log=True)
        param['skip_drop'] = trial.suggest_float('skip_drop', 1e-8, 1.0, log=True)
    
    start_time = time.time()
    
    # Return mean F1 score across all folds
    # Perform stratified cross-validation
    for train_idx, valid_idx in skf.split(X, y):
        # Split data for current fold
        train_x = X[train_idx].to_numpy()
        valid_x = X[valid_idx].to_numpy()
        train_y = y[train_idx].to_numpy().ravel()
        valid_y = y[valid_idx].to_numpy().ravel()

        # Create DMatrix objects
        dtrain = xgb.DMatrix(train_x, label=train_y)
        dvalid = xgb.DMatrix(valid_x, label=valid_y)
        
        # Train model
        bst = xgb.train(param, dtrain)
        
        # Make predictions
        preds = bst.predict(dvalid)
        
        f1, optimal_threshold = find_optimal_f1(valid_y, preds)
        
        cv_scores.append(f1)
        optimal_thresholds.append(float(optimal_threshold))

    execution_time = time.time() - start_time
    
    trial.set_user_attr('execution_time', execution_time)
    trial.set_user_attr('optimal_thresholds', optimal_thresholds)
    trial.set_user_attr('threshold', float(np.mean(optimal_thresholds)))

    # Return mean F1 score across all folds
    return np.mean(cv_scores)

In [56]:
def random_forest_objective(trial, X, y, skf, n_splits=5):
    '''
    Random Forest objective function using stratified cross-validation

    Args:
        trial: Optuna trial object
        X: Feature matrix
        y: Target vector
        skf: Stratified K-Fold cross-validator
        n_splits: Number of folds for cross-validation (default: 5)
    '''

    cv_scores = []

    # Random Forest hyperparameters
    param = {
        'n_jobs': 4,
        'random_state': 42,
        'verbose': 0,

        # Core tree parameters
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.5),

        # Feature sampling parameters
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'max_samples': trial.suggest_float('max_samples', 0.1, 1.0),

        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 1000),

        # Class balancing
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample', None]),
    }

    rf = RandomForestClassifier(**param)

    start_time = time.time()

    for train_idx, valid_idx in skf.split(X, y):
        train_x = X[train_idx].to_numpy()
        valid_x = X[valid_idx].to_numpy()
        train_y = y[train_idx].to_numpy().ravel()
        valid_y = y[valid_idx].to_numpy().ravel()

        rf.fit(train_x, train_y)

        preds = rf.predict(valid_x)

        f1 = f1_score(valid_y, preds)
        cv_scores.append(f1)

    execution_time = time.time() - start_time

    trial.set_user_attr('execution_time', execution_time)

    return np.mean(cv_scores)

In [57]:
def lightgbm_objective(trial, X, y, skf, ratio_negative_to_positive, n_splits=5):
    '''
    LightGBM objective function using stratified cross-validation
    
    Args:
        trial: Optuna trial object
        X: Feature matrix
        y: Target vector
        skf: Stratified K-Fold cross-validator
        n_splits: Number of folds for cross-validation (default: 5)
    '''
    
    cv_scores = []
    optimal_thresholds = []

    # LightGBM hyperparameters
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'verbosity': 0,
        'seed': 42,
        'num_threads': 4,
        'deterministic': True,
        
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'scale_pos_weight': trial.suggest_float("scale_pos_weight", ratio_negative_to_positive * 0.7, ratio_negative_to_positive * 1.5, log=True),

    }
    
    start_time = time.time()

    # Perform stratified cross-validation
    for train_idx, valid_idx in skf.split(X, y):
        # Split data for current fold
        train_x = X[train_idx].to_numpy()
        valid_x = X[valid_idx].to_numpy()
        train_y = y[train_idx].to_numpy().ravel()
        valid_y = y[valid_idx].to_numpy().ravel()
        
        # Create LightGBM datasets
        train_data = lgb.Dataset(train_x, label=train_y)
        valid_data = lgb.Dataset(valid_x, label=valid_y, reference=train_data)
        
        # Train model with early stopping
        model = lgb.train(
            param,
            train_data,
            valid_sets=[valid_data],
            num_boost_round=1000,
            callbacks=[lgb.early_stopping(50, verbose=False), lgb.log_evaluation(0)]
        )
        
        # Make probability predictions
        preds = model.predict(valid_x, num_iteration=model.best_iteration)
        
        f1, optimal_threhsold = find_optimal_f1(valid_y, preds)
        
        cv_scores.append(f1)
        optimal_thresholds.append(optimal_threhsold)

    execution_time = time.time() - start_time
    
    trial.set_user_attr('execution_time', execution_time)
    trial.set_user_attr('optimal_thresholds', optimal_thresholds)
    trial.set_user_attr('threshold', np.mean(optimal_thresholds))
    
    # Return mean F1 score across all folds
    return np.mean(cv_scores)

In [58]:
def catboost_objective(trial, X, y, skf, ratio_negative_to_positive, n_splits=5):
    '''
    CatBoost objective function using stratified cross-validation
    
    Args:
        trial: Optuna trial object
        X: Feature matrix
        y: Target vector
        skf: Stratified K-Fold cross-validator
        n_splits: Number of folds for cross-validation (default: 5)
    '''
    
    cv_scores = []
    optimal_thresholds = []

    # CatBoost hyperparameters (simplified)
    param = {
        'random_seed': 42,
        'verbose': False,
        'allow_writing_files': False,
        'thread_count': 4,
        
        # Core boosting parameters
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        
        # Regularization and overfitting control
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 10.0),
        'scale_pos_weight': trial.suggest_float("scale_pos_weight", ratio_negative_to_positive * 0.7, ratio_negative_to_positive * 1.5, log=True),
        
        # Early stopping
        'early_stopping_rounds': 50,
        'eval_metric': 'F1',
    }
    
    # Create CatBoost classifier
    cb = CatBoostClassifier(**param)
    
    start_time = time.time()

    # Perform stratified cross-validation
    for train_idx, valid_idx in skf.split(X, y):
        # Split data for current fold
        train_x = X[train_idx].to_numpy()
        valid_x = X[valid_idx].to_numpy()
        train_y = y[train_idx].to_numpy().ravel()
        valid_y = y[valid_idx].to_numpy().ravel()
        
        # Train model with validation set for early stopping
        cb.fit(
            train_x, train_y,
            eval_set=(valid_x, valid_y),
            verbose=False
        )
        
        # Make probability predictions
        preds = cb.predict_proba(valid_x)[:, 1]  # Get probability of positive class
    
        f1, optimal_threhsold = find_optimal_f1(valid_y, preds)
        
        cv_scores.append(f1)
        optimal_thresholds.append(optimal_threhsold)

    execution_time = time.time() - start_time
    
    trial.set_user_attr('execution_time', execution_time)
    trial.set_user_attr('optimal_thresholds', optimal_thresholds)
    trial.set_user_attr('threshold', np.mean(optimal_thresholds))

    # Return mean F1 score across all folds
    return np.mean(cv_scores)

In [ ]:
def histgb_objective(trial, X, y, skf, n_splits=5):
    '''
    HistGradientBoostingClassifier objective function using stratified cross-validation

    Args:
        trial: Optuna trial object
        X: Feature matrix
        y: Target vector
        skf: Stratified K-Fold cross-validator
        n_splits: Number of folds for cross-validation (default: 5)
    '''

    cv_scores = []

    # HistGradientBoosting hyperparameters (simplified)
    param = {
        'random_state': 42,
        'verbose': 0,

        # Core boosting parameters
        'max_iter': trial.suggest_int('max_iter', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 100),

        # Regularization
        'l2_regularization': trial.suggest_float('l2_regularization', 0.0, 1.0),
        'max_bins': trial.suggest_int('max_bins', 32, 255),

        # Early stopping
        'early_stopping': True,
        'n_iter_no_change': 10,
        'validation_fraction': 0.1,
    }

    hgb = HistGradientBoostingClassifier(**param)

    start_time = time.time()

    for train_idx, valid_idx in skf.split(X, y):
        train_x = X[train_idx].to_numpy()
        valid_x = X[valid_idx].to_numpy()
        train_y = y[train_idx].to_numpy().ravel()
        valid_y = y[valid_idx].to_numpy().ravel()

        hgb.fit(train_x, train_y)

        preds = hgb.predict(valid_x) 
        f1 = f1_score(valid_y, preds)
        cv_scores.append(f1)

    execution_time = time.time() - start_time

    trial.set_user_attr('execution_time', execution_time)

    return np.mean(cv_scores)


# Start tuning

In [59]:
# Setup
n_trials = 100

## XGBoost

In [60]:
# XGBoost study
xgb_study = optuna.create_study(
    study_name="xgboost_optimization_age_b_1",
    direction="maximize",
    storage=db_dir.format('xgb_study'),
    load_if_exists=True
)
xgb_study.optimize(lambda trial: xgboost_objective(trial, X_1, y_1, skf, ratio_negative_to_positive_b_1), n_trials=n_trials)

print(f"\nBest XGB score: {xgb_study.best_value}")
print(f"Best XGB params: {xgb_study.best_params}")

execution_times = [t.user_attrs.get('execution_time', 0) for t in xgb_study.trials]
print(f"\nAverage execution time XGB: {np.mean(execution_times):.2f}s")
print(f"Total optimization time XGB: {sum(execution_times):.2f}s")

thresholds = xgb_study.best_trial.user_attrs.get('optimal_thresholds', [])
mean_threshold = xgb_study.best_trial.user_attrs.get('threshold', None)
print(f"\nOptimal thresholds (per fold) for best XGB trial: {thresholds}")
print(f"Mean threshold for best XGB trial: {mean_threshold}")

[I 2025-07-01 20:33:42,309] A new study created in RDB with name: xgboost_optimization_age_b_1


[I 2025-07-01 20:33:42,616] Trial 0 finished with value: 0.1524425048454579 and parameters: {'scale_pos_weight': 7.984161063391847, 'booster': 'gblinear', 'n_estimators': 1785, 'learning_rate': 0.167869384753913, 'lambda': 0.006238398022671035, 'alpha': 0.00010691612397185862, 'subsample': 0.41774865476974576, 'colsample_bytree': 0.9463755855789743}. Best is trial 0 with value: 0.1524425048454579.
[I 2025-07-01 20:33:42,851] Trial 1 finished with value: 0.14761179618842574 and parameters: {'scale_pos_weight': 16.716813747443243, 'booster': 'gblinear', 'n_estimators': 1226, 'learning_rate': 0.009412067687142388, 'lambda': 1.6049004194820735e-07, 'alpha': 0.039839063949419364, 'subsample': 0.954224837674676, 'colsample_bytree': 0.21062040170757285}. Best is trial 0 with value: 0.1524425048454579.
[I 2025-07-01 20:33:43,654] Trial 2 finished with value: 0.13798157430301577 and parameters: {'scale_pos_weight': 16.211749844197445, 'booster': 'dart', 'n_estimators': 1899, 'learning_rate': 0.


Best XGB score: 0.16318799437591117
Best XGB params: {'scale_pos_weight': 1.8490891401240388, 'booster': 'dart', 'n_estimators': 298, 'learning_rate': 0.2175665539990347, 'lambda': 1.070284287482838e-06, 'alpha': 2.443988741260388e-08, 'subsample': 0.980210041555195, 'colsample_bytree': 0.6634667101281074, 'max_depth': 3, 'min_child_weight': 4, 'eta': 0.009310315006680062, 'gamma': 2.0644076081618415e-06, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.002734203732356561, 'skip_drop': 4.150607859853549e-05}

Average execution time XGB: 0.72s
Total optimization time XGB: 72.17s

Optimal thresholds (per fold) for best XGB trial: [0.14768944680690765, 0.14368414878845215, 0.1315889209508896, 0.14210809767246246, 0.14151643216609955]
Mean threshold for best XGB trial: 0.1413174092769623


In [61]:
# XGBoost study
xgb_study = optuna.create_study(
    study_name="xgboost_optimization_age_b_2",
    direction="maximize",
    storage=db_dir.format('xgb_study'),
    load_if_exists=True
)
xgb_study.optimize(lambda trial: xgboost_objective(trial, X_2, y_2, skf, ratio_negative_to_positive_b_2), n_trials=n_trials)

print(f"\nBest XGB score: {xgb_study.best_value}")
print(f"Best XGB params: {xgb_study.best_params}")

execution_times = [t.user_attrs.get('execution_time', 0) for t in xgb_study.trials]
print(f"\nAverage execution time XGB: {np.mean(execution_times):.2f}s")
print(f"Total optimization time XGB: {sum(execution_times):.2f}s")

thresholds = xgb_study.best_trial.user_attrs.get('optimal_thresholds', [])
mean_threshold = xgb_study.best_trial.user_attrs.get('threshold', None)
print(f"\nOptimal thresholds (per fold) for best XGB trial: {thresholds}")
print(f"Mean threshold for best XGB trial: {mean_threshold}")

[I 2025-07-01 20:34:58,867] A new study created in RDB with name: xgboost_optimization_age_b_2
[I 2025-07-01 20:34:59,506] Trial 0 finished with value: 0.11728419117263804 and parameters: {'scale_pos_weight': 3.875999566181403, 'booster': 'dart', 'n_estimators': 1254, 'learning_rate': 0.0024376882161510154, 'lambda': 2.0360691685430263e-07, 'alpha': 1.1879096309392409e-08, 'subsample': 0.29926477630364107, 'colsample_bytree': 0.6831803674637287, 'max_depth': 19, 'min_child_weight': 3, 'eta': 0.38779903508849245, 'gamma': 0.0005898454239065433, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.9903380647768043, 'skip_drop': 3.043158219600669e-08}. Best is trial 0 with value: 0.11728419117263804.
[I 2025-07-01 20:35:00,146] Trial 1 finished with value: 0.1182162555370859 and parameters: {'scale_pos_weight': 4.454622061910012, 'booster': 'dart', 'n_estimators': 1475, 'learning_rate': 0.1514758710702782, 'lambda': 6.773461882574638e-05, 'alpha'


Best XGB score: 0.15087501055621325
Best XGB params: {'scale_pos_weight': 14.489749561138987, 'booster': 'dart', 'n_estimators': 1791, 'learning_rate': 0.001207184183290601, 'lambda': 0.018561231411393802, 'alpha': 0.10642542092168038, 'subsample': 0.672715129086682, 'colsample_bytree': 0.7564539530008034, 'max_depth': 5, 'min_child_weight': 4, 'eta': 0.027726198358198426, 'gamma': 2.6629265976902274e-06, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 1.7349811254805127e-07, 'skip_drop': 2.700274707738184e-07}

Average execution time XGB: 0.25s
Total optimization time XGB: 24.73s

Optimal thresholds (per fold) for best XGB trial: [0.44705814123153687, 0.44660136103630066, 0.4449523389339447, 0.4457159638404846, 0.44639208912849426]
Mean threshold for best XGB trial: 0.44614397883415224


## Random Forest

In [62]:
# Random Forest study
rf_study = optuna.create_study(
    study_name="random_forest_optimization_age_b_1",
    direction="maximize",
    storage=db_dir.format('rf_study'),
    load_if_exists=True
)
rf_study.optimize(lambda trial: random_forest_objective(trial, X_1, y_1, skf), n_trials=n_trials)  # Requires too much time to train one, reduce the number of trials

print(f"\nBest RF score: {rf_study.best_value}")
print(f"Best RF params: {rf_study.best_params}")

execution_times = [t.user_attrs.get('execution_time', 0) for t in rf_study.trials]
print(f"\nAverage execution time RF: {np.mean(execution_times):.2f}s")
print(f"Total optimization time RF: {sum(execution_times):.2f}s")

[I 2025-07-01 20:35:29,180] A new study created in RDB with name: random_forest_optimization_age_b_1
[I 2025-07-01 20:35:31,648] Trial 0 finished with value: 0.1554214380723521 and parameters: {'n_estimators': 104, 'max_depth': 20, 'min_samples_split': 18, 'min_samples_leaf': 6, 'min_weight_fraction_leaf': 0.20516041250257794, 'max_features': 'log2', 'max_samples': 0.9192651489057665, 'max_leaf_nodes': 709, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.1554214380723521.
[I 2025-07-01 20:35:43,521] Trial 1 finished with value: 0.15482661926786317 and parameters: {'n_estimators': 710, 'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.23034954893981296, 'max_features': 'sqrt', 'max_samples': 0.5654690400796282, 'max_leaf_nodes': 946, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.1554214380723521.
[I 2025-07-01 20:35:49,355] Trial 2 finished with value: 0.15250130333390527 and parameters: {'n_estimators': 359, 'max_depth'


Best RF score: 0.15949585916468387
Best RF params: {'n_estimators': 414, 'max_depth': 6, 'min_samples_split': 4, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0.0322792826276814, 'max_features': 'sqrt', 'max_samples': 0.23951992376128442, 'max_leaf_nodes': 138, 'class_weight': 'balanced'}

Average execution time RF: 10.00s
Total optimization time RF: 999.66s


In [63]:
# Random Forest study
rf_study = optuna.create_study(
    study_name="random_forest_optimization_age_b_2",
    direction="maximize",
    storage=db_dir.format('rf_study'),
    load_if_exists=True
)
rf_study.optimize(lambda trial: random_forest_objective(trial, X_2, y_2, skf), n_trials=n_trials)  # Requires too much time to train one, reduce the number of trials

print(f"\nBest RF score: {rf_study.best_value}")
print(f"Best RF params: {rf_study.best_params}")

execution_times = [t.user_attrs.get('execution_time', 0) for t in rf_study.trials]
print(f"\nAverage execution time RF: {np.mean(execution_times):.2f}s")
print(f"Total optimization time RF: {sum(execution_times):.2f}s")

[I 2025-07-01 20:52:13,447] A new study created in RDB with name: random_forest_optimization_age_b_2
[I 2025-07-01 20:52:14,649] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 139, 'max_depth': 7, 'min_samples_split': 12, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0.4557167041727808, 'max_features': 'log2', 'max_samples': 0.343841653636192, 'max_leaf_nodes': 554, 'class_weight': None}. Best is trial 0 with value: 0.0.
[I 2025-07-01 20:52:18,399] Trial 1 finished with value: 0.0 and parameters: {'n_estimators': 486, 'max_depth': 6, 'min_samples_split': 17, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0.2396488913335687, 'max_features': 'sqrt', 'max_samples': 0.5261726794293164, 'max_leaf_nodes': 196, 'class_weight': None}. Best is trial 0 with value: 0.0.
[I 2025-07-01 20:52:24,759] Trial 2 finished with value: 0.11636131837452182 and parameters: {'n_estimators': 541, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 10, 'min_weight_fraction_l


Best RF score: 0.1311699270046216
Best RF params: {'n_estimators': 931, 'max_depth': 4, 'min_samples_split': 5, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0.002546502102164078, 'max_features': 'sqrt', 'max_samples': 0.5017851962691026, 'max_leaf_nodes': 286, 'class_weight': 'balanced'}

Average execution time RF: 7.03s
Total optimization time RF: 702.96s


## LightGBM

In [64]:
# LightGBM study
lgb_study = optuna.create_study(
    study_name="lightgbm_optimization_age_b_1",
    direction="maximize",
    storage=db_dir.format('lgb_study'),
    load_if_exists=True
)
lgb_study.optimize(lambda trial: lightgbm_objective(trial, X_1, y_1, skf, ratio_negative_to_positive_b_1), n_trials=n_trials)

print(f"\nBest LightGBM score: {lgb_study.best_value}")
print(f"Best LightGBM params: {lgb_study.best_params}")

execution_times = [t.user_attrs.get('execution_time', 0) for t in lgb_study.trials]
print(f"\nAverage execution time LightGBM: {np.mean(execution_times):.2f}s")
print(f"Total optimization time LightGBM: {sum(execution_times):.2f}s")

thresholds = lgb_study.best_trial.user_attrs.get('optimal_thresholds', [])
mean_threshold = lgb_study.best_trial.user_attrs.get('threshold', None)
print(f"\nOptimal thresholds (per fold) for best LightGBM trial: {thresholds}")
print(f"Mean threshold for best LightGBM trial: {mean_threshold}")

[I 2025-07-01 21:04:01,608] A new study created in RDB with name: lightgbm_optimization_age_b_1
[I 2025-07-01 21:04:06,728] Trial 0 finished with value: 0.1506932547856683 and parameters: {'num_leaves': 86, 'learning_rate': 0.28275127942581596, 'feature_fraction': 0.9437613400892697, 'bagging_fraction': 0.6215798053272861, 'bagging_freq': 6, 'min_child_samples': 45, 'reg_alpha': 0.881033374164938, 'reg_lambda': 0.028726716961458676, 'scale_pos_weight': 16.67819681028466}. Best is trial 0 with value: 0.1506932547856683.
[I 2025-07-01 21:04:42,666] Trial 1 finished with value: 0.13929407033424773 and parameters: {'num_leaves': 247, 'learning_rate': 0.29543032339061825, 'feature_fraction': 0.5708786567487729, 'bagging_fraction': 0.8045309326973792, 'bagging_freq': 6, 'min_child_samples': 51, 'reg_alpha': 0.6485360577499517, 'reg_lambda': 0.49738831457982347, 'scale_pos_weight': 11.909098781622221}. Best is trial 0 with value: 0.1506932547856683.
[I 2025-07-01 21:04:45,830] Trial 2 finishe


Best LightGBM score: 0.15961913510127423
Best LightGBM params: {'num_leaves': 21, 'learning_rate': 0.2020132088190523, 'feature_fraction': 0.7733869325685215, 'bagging_fraction': 0.9067648273393395, 'bagging_freq': 5, 'min_child_samples': 93, 'reg_alpha': 0.7516042979209725, 'reg_lambda': 0.44094257157114747, 'scale_pos_weight': 13.871420499157374}

Average execution time LightGBM: 4.98s
Total optimization time LightGBM: 497.92s

Optimal thresholds (per fold) for best LightGBM trial: [0.26321430253790606, 0.24829233558709723, 0.24092588765079767, 0.22782943699653355, 0.23967813377370442]
Mean threshold for best LightGBM trial: 0.2439880193092078


In [65]:
# LightGBM study
lgb_study = optuna.create_study(
    study_name="lightgbm_optimization_age_b_2",
    direction="maximize",
    storage=db_dir.format('lgb_study'),
    load_if_exists=True
)
lgb_study.optimize(lambda trial: lightgbm_objective(trial, X_2, y_2, skf, ratio_negative_to_positive_b_2), n_trials=n_trials)

print(f"\nBest LightGBM score: {lgb_study.best_value}")
print(f"Best LightGBM params: {lgb_study.best_params}")

execution_times = [t.user_attrs.get('execution_time', 0) for t in lgb_study.trials]
print(f"\nAverage execution time LightGBM: {np.mean(execution_times):.2f}s")
print(f"Total optimization time LightGBM: {sum(execution_times):.2f}s")

thresholds = lgb_study.best_trial.user_attrs.get('optimal_thresholds', [])
mean_threshold = lgb_study.best_trial.user_attrs.get('threshold', None)
print(f"\nOptimal thresholds (per fold) for best LightGBM trial: {thresholds}")
print(f"Mean threshold for best LightGBM trial: {mean_threshold}")

[I 2025-07-01 21:12:26,369] A new study created in RDB with name: lightgbm_optimization_age_b_2


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-07-01 21:12:29,492] Trial 0 finished with value: 0.1104167419464301 and parameters: {'num_leaves': 122, 'learning_rate': 0.047398673951844966, 'feature_fraction': 0.5817539209469433, 'bagging_fraction': 0.4170906634470728, 'bagging_freq': 5, 'min_child_samples': 31, 'reg_alpha': 0.4702220703345956, 'reg_lambda': 0.9539468867417271, 'scale_pos_weight': 17.163966647216014}. Best is trial 0 with value: 0.1104167419464301.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-07-01 21:12:32,958] Trial 1 finished with value: 0.1150954782213516 and parameters: {'num_leaves': 114, 'learning_rate': 0.038553689272154655, 'feature_fraction': 0.7322057514501731, 'bagging_fraction': 0.7431414793933443, 'bagging_freq': 4, 'min_child_samples': 53, 'reg_alpha': 0.7037811456504239, 'reg_lambda': 0.3328491427951181, 'scale_pos_weight': 23.554293120700788}. Best is trial 1 with value: 0.1150954782213516.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-07-01 21:21:39,190] Trial 98 finished with value: 0.12039671842783697 and parameters: {'num_leaves': 119, 'learning_rate': 0.16561336233501162, 'feature_fraction': 0.6890004778941474, 'bagging_fraction': 0.9152004387349746, 'bagging_freq': 2, 'min_child_samples': 24, 'reg_alpha': 0.7477183552806247, 'reg_lambda': 0.3438463684664984, 'scale_pos_weight': 20.010868634553482}. Best is trial 39 with value: 0.13827551517777598.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-07-01 21:21:45,184] Trial 99 finished with value: 0.12459925198702074 and parameters: {'num_leaves': 112, 'learning_rate': 0.13179414894019087, 'feature_fraction': 0.7931108410261902, 'bagging_fraction': 0.8625404895248838, 'bagging_freq': 5, 'min_child_samples': 29, 'reg_alpha': 0.6188659745982864, 'reg_lambda': 0.5800966037449993, 'scale_pos_weight': 20.72822818960656}. Best is trial 39 with value: 0.13827551517777598.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

Best LightGBM score: 0.13827551517777598
Best LightGBM params: {'num_leaves': 80, 'learning_rate': 0.21979530795145358, 'feature_fraction': 0.8174117537984181, 'bagging_fraction': 0.9629170289938864, 'bagging_freq': 4, 'min_child_samples': 43, 'reg_alpha': 0.7755279139054861, 'reg_lambda': 0.5791414013058541, 'scale_pos_weight': 18.82185801404129}

Average execution time LightGBM: 5.53s
Total optimization time LightGBM: 552.92s

Optimal thresholds (per fold) for best LightGBM trial: [0.20704729928377782, 0.13647176879474174, 0.09583739996836785, 0.002978797336177862, 0.145099758966053]
Mean threshold for best LightGB

## Catboost

In [66]:
# CatBoost study
cat_study = optuna.create_study(
    study_name="catboost_optimization_age_b_1",
    direction="maximize",
    storage=db_dir.format('cat_study'),
    load_if_exists=True
)
cat_study.optimize(lambda trial: catboost_objective(trial, X_1, y_1, skf, ratio_negative_to_positive_b_1), n_trials=n_trials)

print(f"\nBest CatBoost score: {cat_study.best_value}")
print(f"Best CatBoost params: {cat_study.best_params}")

execution_times = [t.user_attrs.get('execution_time', 0) for t in cat_study.trials]
print(f"\nAverage execution time CatBoost: {np.mean(execution_times):.2f}s")
print(f"Total optimization time CatBoost: {sum(execution_times):.2f}s")

thresholds = cat_study.best_trial.user_attrs.get('optimal_thresholds', [])
mean_threshold = cat_study.best_trial.user_attrs.get('threshold', None)
print(f"\nOptimal thresholds (per fold) for best CatBoost trial: {thresholds}")
print(f"Mean threshold for best CatBoost trial: {mean_threshold}")

[I 2025-07-01 21:21:45,232] A new study created in RDB with name: catboost_optimization_age_b_1
[I 2025-07-01 21:21:51,345] Trial 0 finished with value: 0.151411944098984 and parameters: {'iterations': 925, 'learning_rate': 0.02544961434651239, 'depth': 8, 'l2_leaf_reg': 8.328232900841115, 'bagging_temperature': 0.6521954214050382, 'random_strength': 7.036475930013167, 'scale_pos_weight': 16.114441474271366}. Best is trial 0 with value: 0.151411944098984.
[I 2025-07-01 21:21:55,328] Trial 1 finished with value: 0.15436418138338298 and parameters: {'iterations': 372, 'learning_rate': 0.016990797213165553, 'depth': 7, 'l2_leaf_reg': 8.129463054003569, 'bagging_temperature': 0.43058878881313944, 'random_strength': 8.035742028504956, 'scale_pos_weight': 14.65483230709347}. Best is trial 1 with value: 0.15436418138338298.
[I 2025-07-01 21:21:58,588] Trial 2 finished with value: 0.15780078830376296 and parameters: {'iterations': 549, 'learning_rate': 0.0444945324884842, 'depth': 7, 'l2_leaf_


Best CatBoost score: 0.1631664990916303
Best CatBoost params: {'iterations': 289, 'learning_rate': 0.017570438392137358, 'depth': 5, 'l2_leaf_reg': 4.1422069039013945, 'bagging_temperature': 0.584124951553705, 'random_strength': 0.038448146342877514, 'scale_pos_weight': 19.07270027359853}

Average execution time CatBoost: 4.69s
Total optimization time CatBoost: 468.88s

Optimal thresholds (per fold) for best CatBoost trial: [0.6088363935948802, 0.5604662954965043, 0.5482290922945263, 0.6297078007279239, 0.5314413501457333]
Mean threshold for best CatBoost trial: 0.5757361864519136


In [67]:
# CatBoost study
cat_study = optuna.create_study(
    study_name="catboost_optimization_age_b_2",
    direction="maximize",
    storage=db_dir.format('cat_study'),
    load_if_exists=True
)
cat_study.optimize(lambda trial: catboost_objective(trial, X_2, y_2, skf, ratio_negative_to_positive_b_2), n_trials=n_trials)

print(f"\nBest CatBoost score: {cat_study.best_value}")
print(f"Best CatBoost params: {cat_study.best_params}")

execution_times = [t.user_attrs.get('execution_time', 0) for t in cat_study.trials]
print(f"\nAverage execution time CatBoost: {np.mean(execution_times):.2f}s")
print(f"Total optimization time CatBoost: {sum(execution_times):.2f}s")

thresholds = cat_study.best_trial.user_attrs.get('optimal_thresholds', [])
mean_threshold = cat_study.best_trial.user_attrs.get('threshold', None)
print(f"\nOptimal thresholds (per fold) for best CatBoost trial: {thresholds}")
print(f"Mean threshold for best CatBoost trial: {mean_threshold}")

[I 2025-07-01 21:29:38,194] A new study created in RDB with name: catboost_optimization_age_b_2
[I 2025-07-01 21:29:39,611] Trial 0 finished with value: 0.13552584542719676 and parameters: {'iterations': 940, 'learning_rate': 0.08906080018010061, 'depth': 8, 'l2_leaf_reg': 3.3080026642703078, 'bagging_temperature': 0.07565468883394633, 'random_strength': 5.149526057430088, 'scale_pos_weight': 20.0595459020457}. Best is trial 0 with value: 0.13552584542719676.
[I 2025-07-01 21:29:40,838] Trial 1 finished with value: 0.13596523565175897 and parameters: {'iterations': 367, 'learning_rate': 0.010912675091097543, 'depth': 5, 'l2_leaf_reg': 4.52964840859399, 'bagging_temperature': 0.31001988952353243, 'random_strength': 1.927966565084669, 'scale_pos_weight': 26.41691346765666}. Best is trial 1 with value: 0.13596523565175897.
[I 2025-07-01 21:29:42,212] Trial 2 finished with value: 0.1325589248312043 and parameters: {'iterations': 525, 'learning_rate': 0.05595824182507507, 'depth': 8, 'l2_le


Best CatBoost score: 0.15797597987727255
Best CatBoost params: {'iterations': 699, 'learning_rate': 0.07868461607441338, 'depth': 7, 'l2_leaf_reg': 9.63938710311811, 'bagging_temperature': 0.49506113817595054, 'random_strength': 0.7251009782304432, 'scale_pos_weight': 20.39226476767552}

Average execution time CatBoost: 0.96s
Total optimization time CatBoost: 95.97s

Optimal thresholds (per fold) for best CatBoost trial: [0.5335137159859573, 0.5169955449994765, 0.5567414823998357, 0.5168239654531086, 0.5607729781292473]
Mean threshold for best CatBoost trial: 0.536969537393525


## HistGradientBoosting

In [71]:
# HistGradientBoosting study
histgb_study = optuna.create_study(
    study_name="histgb_optimization_age_b_1",
    direction="maximize",
    storage=db_dir.format('histgb_study'),
    load_if_exists=True
)
histgb_study.optimize(lambda trial: histgb_objective(trial, X_1, y_1, skf), n_trials=n_trials)

print(f"\nBest HistGB score: {histgb_study.best_value}")
print(f"Best HistGB params: {histgb_study.best_params}")

execution_times = [t.user_attrs.get('execution_time', 0) for t in histgb_study.trials]
print(f"\nAverage execution time HistGB: {np.mean(execution_times):.2f}s")
print(f"Total optimization time HistGB: {sum(execution_times):.2f}s")

thresholds = histgb_study.best_trial.user_attrs.get('optimal_thresholds', [])
mean_threshold = histgb_study.best_trial.user_attrs.get('threshold', None)
print(f"\nOptimal thresholds (per fold) for best HistGB trial: {thresholds}")
print(f"Mean threshold for best HistGB trial: {mean_threshold}")

[I 2025-07-01 22:56:59,491] A new study created in RDB with name: histgb_optimization_age_b_1
[I 2025-07-01 22:57:00,239] Trial 0 finished with value: 0.16180670560687382 and parameters: {'max_iter': 231, 'learning_rate': 0.24424668172093783, 'max_depth': 3, 'min_samples_leaf': 23, 'l2_regularization': 0.7544680299396979, 'max_bins': 121}. Best is trial 0 with value: 0.16180670560687382.
[I 2025-07-01 22:57:01,306] Trial 1 finished with value: 0.15690360728713326 and parameters: {'max_iter': 200, 'learning_rate': 0.17514744302736057, 'max_depth': 8, 'min_samples_leaf': 26, 'l2_regularization': 0.16712287774578183, 'max_bins': 182}. Best is trial 0 with value: 0.16180670560687382.
[I 2025-07-01 22:57:04,384] Trial 2 finished with value: 0.16057210893693802 and parameters: {'max_iter': 159, 'learning_rate': 0.02558241028659679, 'max_depth': 5, 'min_samples_leaf': 94, 'l2_regularization': 0.99470985709467, 'max_bins': 207}. Best is trial 0 with value: 0.16180670560687382.
[I 2025-07-01 22


Best HistGB score: 0.1635457725323755
Best HistGB params: {'max_iter': 157, 'learning_rate': 0.09137833371402335, 'max_depth': 3, 'min_samples_leaf': 98, 'l2_regularization': 0.25018585110288427, 'max_bins': 173}

Average execution time HistGB: 1.90s
Total optimization time HistGB: 189.53s

Optimal thresholds (per fold) for best HistGB trial: [0.07437066181268026, 0.07778512063221418, 0.08680480075314145, 0.08206459212665583, 0.08030071354920651]
Mean threshold for best HistGB trial: 0.08026517777477964


In [72]:
# HistGradientBoosting study
histgb_study = optuna.create_study(
    study_name="histgb_optimization_age_b_2",
    direction="maximize",
    storage=db_dir.format('histgb_study'),
    load_if_exists=True
)
histgb_study.optimize(lambda trial: histgb_objective(trial, X_2, y_2, skf), n_trials=n_trials)

print(f"\nBest HistGB score: {histgb_study.best_value}")
print(f"Best HistGB params: {histgb_study.best_params}")

execution_times = [t.user_attrs.get('execution_time', 0) for t in histgb_study.trials]
print(f"\nAverage execution time HistGB: {np.mean(execution_times):.2f}s")
print(f"Total optimization time HistGB: {sum(execution_times):.2f}s")

thresholds = histgb_study.best_trial.user_attrs.get('optimal_thresholds', [])
mean_threshold = histgb_study.best_trial.user_attrs.get('threshold', None)
print(f"\nOptimal thresholds (per fold) for best HistGB trial: {thresholds}")
print(f"Mean threshold for best HistGB trial: {mean_threshold}")

[I 2025-07-01 23:00:13,256] A new study created in RDB with name: histgb_optimization_age_b_2
[I 2025-07-01 23:00:16,401] Trial 0 finished with value: 0.12401049216597874 and parameters: {'max_iter': 291, 'learning_rate': 0.0028702205188732696, 'max_depth': 6, 'min_samples_leaf': 19, 'l2_regularization': 0.04555690680234614, 'max_bins': 181}. Best is trial 0 with value: 0.12401049216597874.
[I 2025-07-01 23:00:16,959] Trial 1 finished with value: 0.1327257162116548 and parameters: {'max_iter': 383, 'learning_rate': 0.21540493409214456, 'max_depth': 6, 'min_samples_leaf': 73, 'l2_regularization': 0.6981161340313383, 'max_bins': 243}. Best is trial 1 with value: 0.1327257162116548.
[I 2025-07-01 23:00:20,609] Trial 2 finished with value: 0.1171148841737077 and parameters: {'max_iter': 472, 'learning_rate': 0.003704221446184845, 'max_depth': 7, 'min_samples_leaf': 94, 'l2_regularization': 0.6758497112645396, 'max_bins': 198}. Best is trial 1 with value: 0.1327257162116548.
[I 2025-07-01 2


Best HistGB score: 0.1452386253041233
Best HistGB params: {'max_iter': 323, 'learning_rate': 0.2053520596646573, 'max_depth': 4, 'min_samples_leaf': 25, 'l2_regularization': 0.25776573696908545, 'max_bins': 123}

Average execution time HistGB: 0.68s
Total optimization time HistGB: 67.87s

Optimal thresholds (per fold) for best HistGB trial: [0.07255081379937263, 0.08841334514404836, 0.06557626317399529, 0.055622585809771655, 0.06735443810320485]
Mean threshold for best HistGB trial: 0.06990348920607856


# Conclusion

In [73]:
xgb_study = optuna.load_study(
    study_name="xgboost_optimization_age_b_1",
    storage=db_dir.format('xgb_study')
)
xgb_study_b2 = optuna.load_study(
    study_name="xgboost_optimization_age_b_2",
    storage=db_dir.format('xgb_study')
)
rf_study = optuna.load_study(
    study_name="random_forest_optimization_age_b_1",
    storage=db_dir.format('rf_study')
)
rf_study_b2 = optuna.load_study(
    study_name="random_forest_optimization_age_b_2",
    storage=db_dir.format('rf_study')
)
lgb_study = optuna.load_study(
    study_name="lightgbm_optimization_age_b_1",
    storage=db_dir.format('lgb_study')
)
lgb_study_b2 = optuna.load_study(
    study_name="lightgbm_optimization_age_b_2",
    storage=db_dir.format('lgb_study')
)
cat_study = optuna.load_study(
    study_name="catboost_optimization_age_b_1",
    storage=db_dir.format('cat_study')
)
cat_study_b2 = optuna.load_study(
    study_name="catboost_optimization_age_b_2",
    storage=db_dir.format('cat_study')
)
histgb_study = optuna.load_study(
    study_name="histgb_optimization_age_b_1",
    storage=db_dir.format('histgb_study')
)
histgb_study_b2 = optuna.load_study(
    study_name="histgb_optimization_age_b_2",
    storage=db_dir.format('histgb_study')
)


In [74]:
print("XGBoost age_b_1 best score:", xgb_study.best_value)
print("XGBoost age_b_2 best score:", xgb_study_b2.best_value)

print("Random Forest age_b_1 best score:", rf_study.best_value)
print("Random Forest age_b_2 best score:", rf_study_b2.best_value)

print("LightGBM age_b_1 best score:", lgb_study.best_value)
print("LightGBM age_b_2 best score:", lgb_study_b2.best_value)

print("CatBoost age_b_1 best score:", cat_study.best_value)
print("CatBoost age_b_2 best score:", cat_study_b2.best_value)

print("HistGB age_b_1 best score:", histgb_study.best_value)
print("HistGB age_b_2 best score:", histgb_study_b2.best_value)

XGBoost age_b_1 best score: 0.16318799437591117
XGBoost age_b_2 best score: 0.15087501055621325
Random Forest age_b_1 best score: 0.15949585916468387
Random Forest age_b_2 best score: 0.1311699270046216
LightGBM age_b_1 best score: 0.15961913510127423
LightGBM age_b_2 best score: 0.13827551517777598
CatBoost age_b_1 best score: 0.1631664990916303
CatBoost age_b_2 best score: 0.15797597987727255
HistGB age_b_1 best score: 0.1635457725323755
HistGB age_b_2 best score: 0.1452386253041233
